# Chapter 9: Load Flow Studies

## Example 9.10: solve_using_gauss_seidel_for_2_variables.sce

In [ ]:
clear
clc

x=.5
y=.5
for i=1:4
    x=(.7*sin(x))+(.2*cos(y))
    y=(.7*cos(x))-(.2*sin(y))
    
end

mprintf('x= %f, y=%f',x,y)

## Example 9.11: find_bus_voltage_and_load_angle_using_GS.sce

In [ ]:
clear
clc

ey=1/(.05 + %i*.15)
y=[
1 2 ey
1 5 ey
2 5 ey
2 3 ey
3 4 ey
4 5 ey
]
n=5
e=6

Y=zeros(n,n)

for i=1:e
    n1=real(y(i,1))
    n2=real(y(i,2))
    ynew=y(i,3)
    if(n1==0)
        Y(n2,n2)=Y(n2,n2)+ynew
    else
        Y(n1,n1)= Y(n1,n1)+ynew
        Y(n1,n2)= Y(n1,n2)-ynew
        Y(n2,n1)= Y(n2,n1)-ynew
        Y(n2,n2)= Y(n2,n2)+ynew
    end
end
// bus no | PL | QL | PG | QG | V | th | btype | Qmin | Qmax

data = [
1 1 .5 %inf %inf 1.02 0 1 %inf %inf
2 0 0 2 %inf 1.02 0 2 .2 .6
3 .5 .2 0 0 1 0   3 %inf %inf
4 .5 .2 0 0 1 0   3 %inf %inf
5 .5 .2 0 0 1 0   3 %inf %inf

]
disp(Y, '(a)')
j = data(:,1);    // Bus number.
PL = data(:,2);
QL = data(:,3);
PG = data(:,4);
QG = data(:,5);
V = data(:,6); 
th = data(:,7);
btype = data(:,8);   // Type of Bus 1-Slack, 2-PV, 3-PQ.
Qmin = data(:,9);
Qmax = data(:,10);
n = max(j);
P = PG - PL;
Q = QG - QL

for i=1:n
    V(i,1)=V(i,1) * exp (%i * th(i,1)* %pi / 180)
end

Vprev = V;
toler = 1000;
iteration = 1;
while (iteration==1)
    for i = 2:n
        summ = 0;
        for k = 1:n
            if i ~= k
                summ = summ + Y(i,k)* V(k);
            end
        end
        if btype(i) == 2
            Q(i) = -imag(conj(V(i))*(summ + Y(i,i)*V(i)));
            if (Q(i) > Qmax(i)) | (Q(i) < Qmin(i))
                if Q(i) < Qmin(i)
                    Q(i) = Qmin(i);
                else    
                    Q(i) = Qmax(i);
                end
                btype(i) = 3;
            end
        end
        V(i) = (1/Y(i,i))*(((P(i)- %i*Q(i))/conj(V(i))) - summ);
        if btype(i) == 2 
            V(i) = abs(Vprev(i))*exp(%i * atan(imag(V(i))/real(V(i))));
        end
    end
    iteration = iteration + 1;
    toler = max(abs(abs(V) - abs(Vprev)));
    Vprev = V; 
end 
disp('(b)')
mprintf('
V3 = %.2f ang(%.2f) deg', abs(V(3)),atand(imag(V(3))/real(V(3))))
mprintf('
V4 = %.3f ang(%.2f) deg', abs(V(4)),atand(imag(V(4))/real(V(4))))
mprintf('
V5 = %.4f ang(%.2f) deg', abs(V(5)),atand(imag(V(5))/real(V(5))))
mprintf('
delta 2 = %.2f deg',atand(imag(V(2,1))/real(V(2,1))))
mprintf('
Q2 = %.4f ',Q(2,1))


## Example 9.12: find_bus_voltage_and_load_angle_using_GS_minimum_value_of_Q2_given.sce

In [ ]:
clear
clc

ey=1/(.05 + %i*.15)
y=[
1 2 ey
1 5 ey
2 5 ey
2 3 ey
3 4 ey
4 5 ey
]
n=5
e=6

Y=zeros(n,n)

for i=1:e
    n1=real(y(i,1))
    n2=real(y(i,2))
    ynew=y(i,3)
    if(n1==0)
        Y(n2,n2)=Y(n2,n2)+ynew
    else
        Y(n1,n1)= Y(n1,n1)+ynew
        Y(n1,n2)= Y(n1,n2)-ynew
        Y(n2,n1)= Y(n2,n1)-ynew
        Y(n2,n2)= Y(n2,n2)+ynew
    end
end
// bus no | PL | QL | PG | QG | V | th | btype | Qmin | Qmax

data = [
1 1 .5 %inf %inf 1.02 0 1 %inf %inf
2 0 0 2 %inf 1.02 0 2 .3 %inf
3 .5 .2 0 0 1 0   3 %inf %inf
4 .5 .2 0 0 1 0   3 %inf %inf
5 .5 .2 0 0 1 0   3 %inf %inf

]

j = data(:,1);    // Bus number.
PL = data(:,2);
QL = data(:,3);
PG = data(:,4);
QG = data(:,5);
V = data(:,6); 
th = data(:,7);
btype = data(:,8);   // Type of Bus 1-Slack, 2-PV, 3-PQ.
Qmin = data(:,9);
Qmax = data(:,10);
n = max(j);
P = PG - PL;
Q = QG - QL

for i=1:n
    V(i,1)=V(i,1) * exp (%i * th(i,1)* %pi / 180)
end

Vprev = V;
toler = 1000;
iteration = 1;
while (iteration==1)
    for i = 2:n
        summ = 0;
        for k = 1:n
            if i ~= k
                summ = summ + Y(i,k)* V(k);
            end
        end
        if btype(i) == 2
            Q(i) = -imag(conj(V(i))*(summ + Y(i,i)*V(i)));
            if (Q(i) > Qmax(i)) | (Q(i) < Qmin(i))
                if Q(i) < Qmin(i)
                    Q(i) = Qmin(i);
                else    
                    Q(i) = Qmax(i);
                end
                btype(i) = 3;
            end
        end
        V(i) = (1/Y(i,i))*(((P(i)- %i*Q(i))/conj(V(i))) - summ);
        if btype(i) == 2 
            V(i) = abs(Vprev(i))*exp(%i * atan(imag(V(i))/real(V(i))));
        end
    end
    iteration = iteration + 1;
    toler = max(abs(abs(V) - abs(Vprev)));
    Vprev = V; 
end 
V=round(V*1e3)/1e3
ansmat(:,1)=[1;2;3;4;5]
ansmat(:,2)=round(abs(V(:,1))*1000)/1000
for i=1:5
    ansmat(i,3)=round(atand(imag(V(i))/real(V(i)))*1e3)/1e3
end
disp(ansmat,' bus no  |V|      delta')

## Example 9.13: solve_using_newton_raphson_1_variable.sce

In [ ]:
clear
clc
E=10
x=1
e=1e-5

while (E>e)
    f=(2*x) - (.43429 * log(x))-7
    df=2-(.43429 /x)
    x1=x-(f/df)
    E=abs(x1-x)
    x=x1
end
mprintf('x=%f',x)

## Example 9.14: solve_using_newton_raphson_2_variables.sce

In [ ]:
clear
clc

E=10
x=3.4
y=2.2
e=1e-5

while (E>e)
    X=[x;y]
    f=(x) + (3* .43429 * log(x))-y^2
    dfx=1 + (3* .43429 /x)
    dfy=-2*y
    g=(2*x*x) - (x*y)-(5*x)+1
    dgx=(4*x) - (y)-(5)
    dgy=-x
    J=[dfx dfy; dgx dgy]
    F=[f;g]
    X1=X-(inv(J)* F)
    E=max(abs(X1-X))
    x=X1(1,1)
    y=X1(2,1)
end
mprintf('x= %.4f, y=%.4f', x,y)


## Example 9.15: solve_using_newton_raphson_1_variable.sce

In [ ]:
clear
clc
E=10
x=0
e=1e-4

while (E>e)
    f=(3*x) - (cos(x))-1
    df=3+sin(x)
    x1=x-(f/df)
    E=abs(x1-x)
    x=x1
end
mprintf('x=%.4f',x)

## Example 9.17: solve_system_using_newton_raphson_method.sce

In [ ]:
clear
clc

y=[
1 2 .026+%i*.11 %i*.04
2 3 .026+%i*.11 %i*.04
1 3 .026+%i*.11 %i*.04
]


n=3
e=3
Y=zeros(n,n)

for i=1:e
    ynew = 1/(y(i,3))
    y0=y(i,4)/2
    n1=real(y(i,1))
    n2=real(y(i,2))
    if(n1==0)
        Y(n2,n2)=Y(n2,n2)+ynew+y0
    else
        Y(n1,n1)= Y(n1,n1)+ynew+y0
        Y(n1,n2)= Y(n1,n2)-ynew
        Y(n2,n1)= Y(n2,n1)-ynew
        Y(n2,n2)= Y(n2,n2)+ynew+y0
    end
end
for i=1:n
    for j=1:n
        if i==j then
            Yb(i,j)= string(round(abs(Y(i,j))*1000)/1000) +'/_'+ string(round(atand(imag(Y(i,j))/real(Y(i,j)))*100)/100)
        else
            Yb(i,j)= string(round(abs(Y(i,j))*1000)/1000) +'/_'+ string(round((atand(imag(Y(i,j))/real(Y(i,j)))+180)*100)/100)
        end
    end
end
Y=round(Y*1e3)/1e3
disp(Yb,'(a)Ybus')
// bus no | PL | QL | PG | QG | V | th | btype 

data = [
1 %inf %inf 1 .5  1.03 0 1 
2 1.5 %inf  0 0 1.03 0 2 
3 0 0  1.2 .5  1 0   3 
]

j = data(:,1);    // Bus number.
PG = data(:,2);
QG = data(:,3);
PL = data(:,4);
QL = data(:,5);
V = data(:,6); 
th = data(:,7)* %pi / 180;
btype = data(:,8);   // Type of Bus 1-Slack, 2-PV, 3-PQ.
nbus = max(j);
P = PG - PL;
Q = QG - QL
Psp = P;
Qsp = Q;
G = real(Y);
B = imag(Y);
for i=1:nbus
    V(i,1)=V(i,1) * exp (%i * th(i,1))
end

pv = [2]        //whicih bus is PV
pq = [3]        //whicih bus is PQ
npv = length(pv);
npq = length(pq);

Tol = 1;  
Iter = 1;
while (Iter == 1)
    
    P = zeros(nbus,1);
    Q = zeros(nbus,1);

    for i = 1:nbus
        for k = 1:nbus
            P(i) = P(i) + V(i)* V(k)*(G(i,k)*cos(th(i)-th(k)) + B(i,k)*sin(th(i)-th(k)));
            Q(i) = Q(i) + V(i)* V(k)*(G(i,k)*sin(th(i)-th(k)) - B(i,k)*cos(th(i)-th(k)));
        end
    end

    dPa = Psp-P;
    dQa = Qsp-Q;
    k = 1;
    dQ = zeros(npq,1);
    for i = 1:nbus
        if btype(i) == 3
            dQ(k,1) = dQa(i);
            k = k+1;
        end
    end
    dP = dPa(2:nbus);
    M = [dP; dQ];

    H = zeros(nbus-1,nbus-1);
    for i = 1:(nbus-1)
        m = i+1;
        for k = 1:(nbus-1)
            n = k+1;
            if n == m
                for n = 1:nbus
                    H(i,k) = H(i,k) + V(m)* V(n)*(-G(m,n)*sin(th(m)-th(n)) + B(m,n)*cos(th(m)-th(n)));
                end
                H(i,k) = H(i,k) - V(m)^2*B(m,m);
            else
                H(i,k) = V(m)* V(n)*(G(m,n)*sin(th(m)-th(n)) - B(m,n)*cos(th(m)-th(n)));
            end
        end
    end
    
    N = zeros(nbus-1,npq);
    for i = 1:(nbus-1)
        m = i+1;
        for k = 1:npq
            n = pq(k);
            if n == m
                for n = 1:nbus
                    N(i,k) = N(i,k) + V(n)*(G(m,n)*cos(th(m)-th(n)) + B(m,n)*sin(th(m)-th(n)));
                end
                N(i,k) = N(i,k) + V(m)*G(m,m);
            else
                N(i,k) = V(m)*(G(m,n)*cos(th(m)-th(n)) + B(m,n)*sin(th(m)-th(n)));
            end
        end
    end
    
    J3 = zeros(npq,nbus-1);
    for i = 1:npq
        m = pq(i);
        for k = 1:(nbus-1)
            n = k+1;
            if n == m
                for n = 1:nbus
                    J3(i,k) = J3(i,k) + V(m)* V(n)*(G(m,n)*cos(th(m)-th(n)) + B(m,n)*sin(th(m)-th(n)));
                end
                J3(i,k) = J3(i,k) - V(m)^2*G(m,m);
            else
                J3(i,k) = V(m)* V(n)*(-G(m,n)*cos(th(m)-th(n)) - B(m,n)*sin(th(m)-th(n)));
            end
        end
    end
    
    L = zeros(npq,npq);
    for i = 1:npq
        m = pq(i);
        for k = 1:npq
            n = pq(k);
            if n == m
                for n = 1:nbus
                    L(i,k) = L(i,k) + V(n)*(G(m,n)*sin(th(m)-th(n)) - B(m,n)*cos(th(m)-th(n)));
                end
                L(i,k) = L(i,k) - V(m)*B(m,m);
            else
                L(i,k) = V(m)*(G(m,n)*sin(th(m)-th(n)) - B(m,n)*cos(th(m)-th(n)));
            end
        end
    end
    
    J = [H N; J3 L];
    
    X = inv(J)*M;
    dTh = X(1:npq+npv);
    dV = X(nbus:nbus+npq-1);
    

    th(2:nbus) = dTh + th(2:nbus);
    k = 1;
    for i = 2:nbus
        if btype(i) == 3
            V(i) = dV(k) + V(i);
            k = k+1;
        end
    end
    
    Iter = Iter + 1;
    Tol = max(abs(M));
    
end
mprintf('

(b)P2= %.3f, P2= %.3f, P3= %.3f, Q3= %.3f', P(2),Q(2) ,P(3), Q(3))
J=fix(J*1e3)/1e3
disp(J, '(c)J')

mprintf('
(d)
')
mprintf('%.3f = %.3f dd2 + %.3f dd3 +%.3f dV3/|V3|
', dP(1), J(1,1), J(1,2), J(1,3) )
mprintf('%.3f = %.3f dd2 + %.3f dd3 +%.3f dV3/|V3|
', dP(2), J(2,1), J(2,2), J(2,3) )
mprintf('%.3f = %.3f dd2 + %.3f dd3 +%.3f dV3/|V3|
', dQ(1), J(3,1), J(3,2), J(3,3) )

## Example 9.18: solve_system_using_fast_decoupled_method.sce

In [ ]:
clear
clc

y=[
1 2 %i*.11 
2 3 %i*.11 
1 3 %i*.11 
]


n=3
e=3
Y=zeros(n,n)

for i=1:e
    ynew = 1/(y(i,3))
    n1=real(y(i,1))
    n2=real(y(i,2))
    if(n1==0)
        Y(n2,n2)=Y(n2,n2)+ynew
    else
        Y(n1,n1)= Y(n1,n1)+ynew
        Y(n1,n2)= Y(n1,n2)-ynew
        Y(n2,n1)= Y(n2,n1)-ynew
        Y(n2,n2)= Y(n2,n2)+ynew
    end
end
Y=round(Y*1e3)/1e3
disp(Y,'(a)Ybus')
// bus no | PL | QL | PG | QG | V | th | btype 

data = [
1 %inf %inf 1 .5  1.03 0 1 
2 1.5 %inf  0 0 1.03 0 2 
3 0 0  1.2 .5  1 0   3 
]

j = data(:,1);    // Bus number.
PG = data(:,2);
QG = data(:,3);
PL = data(:,4);
QL = data(:,5);
V = data(:,6); 
th = data(:,7)* %pi / 180;
btype = data(:,8);   // Type of Bus 1-Slack, 2-PV, 3-PQ.
nbus = max(j);
P = PG - PL;
Q = QG - QL
Psp = P;
Qsp = Q;
G = real(Y);
B = imag(Y);
for i=1:nbus
    V(i,1)=V(i,1) * exp (%i * th(i,1))
end

pv = [2]        //whicih bus is PV
pq = [3]        //whicih bus is PQ
npv = length(pv);
npq = length(pq);

Tol = 1;  
Iter = 1;
while (Iter == 1)
    
    P = zeros(nbus,1);
    Q = zeros(nbus,1);

    for i = 1:nbus
        for k = 1:nbus
            P(i) = P(i) + V(i)* V(k)*(G(i,k)*cos(th(i)-th(k)) + B(i,k)*sin(th(i)-th(k)));
            Q(i) = Q(i) + V(i)* V(k)*(G(i,k)*sin(th(i)-th(k)) - B(i,k)*cos(th(i)-th(k)));
        end
    end

    dPa = Psp-P;
    dQa = Qsp-Q;
    k = 1;
    dQ = zeros(npq,1);
    for i = 1:nbus
        if btype(i) == 3
            dQ(k,1) = dQa(i);
            k = k+1;
        end
    end
    dP = dPa(2:nbus);
    M = [dP; dQ];

    H = zeros(nbus-1,nbus-1);
    for i = 1:(nbus-1)
        m = i+1;
        for k = 1:(nbus-1)
            n = k+1;
            if n == m
                for n = 1:nbus
                    H(i,k) = H(i,k) + V(m)* V(n)*(-G(m,n)*sin(th(m)-th(n)) + B(m,n)*cos(th(m)-th(n)));
                end
                H(i,k) = H(i,k) - V(m)^2*B(m,m);
            else
                H(i,k) = V(m)* V(n)*(G(m,n)*sin(th(m)-th(n)) - B(m,n)*cos(th(m)-th(n)));
            end
        end
    end
    
    N = zeros(nbus-1,npq);
    for i = 1:(nbus-1)
        m = i+1;
        for k = 1:npq
            n = pq(k);
            if n == m
                for n = 1:nbus
                    N(i,k) = N(i,k) + V(n)*(G(m,n)*cos(th(m)-th(n)) + B(m,n)*sin(th(m)-th(n)));
                end
                N(i,k) = N(i,k) + V(m)*G(m,m);
            else
                N(i,k) = V(m)*(G(m,n)*cos(th(m)-th(n)) + B(m,n)*sin(th(m)-th(n)));
            end
        end
    end
    
    J3 = zeros(npq,nbus-1);
    for i = 1:npq
        m = pq(i);
        for k = 1:(nbus-1)
            n = k+1;
            if n == m
                for n = 1:nbus
                    J3(i,k) = J3(i,k) + V(m)* V(n)*(G(m,n)*cos(th(m)-th(n)) + B(m,n)*sin(th(m)-th(n)));
                end
                J3(i,k) = J3(i,k) - V(m)^2*G(m,m);
            else
                J3(i,k) = V(m)* V(n)*(-G(m,n)*cos(th(m)-th(n)) - B(m,n)*sin(th(m)-th(n)));
            end
        end
    end
    
    L = zeros(npq,npq);
    for i = 1:npq
        m = pq(i);
        for k = 1:npq
            n = pq(k);
            if n == m
                for n = 1:nbus
                    L(i,k) = L(i,k) + V(n)*(G(m,n)*sin(th(m)-th(n)) - B(m,n)*cos(th(m)-th(n)));
                end
                L(i,k) = L(i,k) - V(m)*B(m,m);
            else
                L(i,k) = V(m)*(G(m,n)*sin(th(m)-th(n)) - B(m,n)*cos(th(m)-th(n)));
            end
        end
    end
    
    J = [H N; J3 L];
    
    X = inv(J)*M;
    dTh = X(1:npq+npv);
    dV = X(nbus:nbus+npq-1);
    

    th(2:nbus) = dTh + th(2:nbus);
    k = 1;
    for i = 2:nbus
        if btype(i) == 3
            V(i) = -dV(k) + V(i);
            k = k+1;
        end
    end
    
    Iter = Iter + 1;
    Tol = max(abs(M));
    
end

V=round(V*10000)/10000
th=round(th*1e5)/1e5

mprintf('
(b)P2=%.0f P3=%.0f Q3=%.3f 
', P(2), P(3), Q(3))
disp(-imag(Y(2:3,2:3)), '(c)B''')
disp(-imag(Y(3,3)), 'B''''')

P = zeros(nbus,1);
Q = zeros(nbus,1);

for i = 1:nbus
    for k = 1:nbus
        P(i) = P(i) + real(V(i)* V(k)*Y(i,k)* exp(%i * (th(k)-th(i))))
        Q(i) = Q(i) - imag(V(i)* V(k)*Y(i,k)* exp(%i * (th(k)-th(i))))
    end
end
//'please note: there is an error in the book in calculation of Q3=-.0497 (part(e)) during value substitution in formula vi. The variation in results is due to the same error, verified seperately on calculator'
mprintf('

(e)P2= %.3f, P3= %.3f, Q3= %.3f', P(2), P(3), Q(3))

Q(3)=-.49
Pl=P(1)+P(2)+P(3)
Ql=Q(1)+Q(2)+Q(3)


mprintf('

(f)real line losses= %.1f, reactive line losses= %.1f', Pl, Ql)


data(1,2)=PL(1)+P(1);
data(1,3)=QL(1)+Q(1);
data(2,3)=QL(2)+Q(2);


mprintf('

(g)PG1= %.1f,QG1= %.2f, PG2= %.1f,QG2= %.2f

', data(1,2), data(1,3),data(2,2), data(2,3))

disp('please note: there is an error in the book in calculation of Q3=-.0497 (part(e)) during value substitution in formula vi. The variation in results is due to the same error, verified seperately on calculator')

## Example 9.19: solve_system_using_gauss_seidel_method_with_acceleration_constant.sce

In [ ]:
clear
clc

y=[
1 2 2-%i*8
1 3 1-%i*4
2 3 .666-%i*2.664
2 4 1-%i*4
3 4 2-%i*8
]
n=max(real(y(:,1:2)))
e=5

Y=zeros(n,n)

for i=1:e
    n1=real(y(i,1))
    n2=real(y(i,2))
    ynew=y(i,3)
    if(n1==0)
        Y(n2,n2)=Y(n2,n2)+ynew
    else
        Y(n1,n1)= Y(n1,n1)+ynew
        Y(n1,n2)= Y(n1,n2)-ynew
        Y(n2,n1)= Y(n2,n1)-ynew
        Y(n2,n2)= Y(n2,n2)+ynew
    end
end

disp(Y,'Ybus')
// bus no | P | Q | V | th | btype 

data = [
1 %inf %inf 1.06 0 1
2 .5 .2 1 0   3
3 .4 .3 1 0   3 
4 .3 .1 1 0   3 
]

j = data(:,1);    // Bus number.
P = data(:,2);
Q = data(:,3);
V = data(:,4); 
th = data(:,5);
btype = data(:,6);   // Type of Bus 1-Slack, 2-PV, 3-PQ.
n = max(j);

a=1.6

for i=1:n
    V(i,1)=V(i,1) * exp (%i * th(i,1)* %pi / 180)
end

Vprev = V;
toler = 1000;
iteration = 1;
disp('(b)')
while (iteration==1)
    for i = 2:n
        summ = 0;
        for k = 1:n
            if i ~= k
                summ = summ + Y(i,k)* V(k);
            end
        end
        V(i) = (1/Y(i,i))*(((P(i)- %i*Q(i))/conj(V(i))) - summ);
        dv=V(i)-Vprev(i)
        mprintf('
V%d = %.3f ang(%.2f) deg', i,abs(V(i)),atand(imag(V(i))/real(V(i))))
        V(i)=Vprev(i)+(a*dv)
        if btype(i) == 2 
            V(i) = abs(Vprev(i))*exp(%i * atan(imag(V(i))/real(V(i))));
        end
    end
    iteration = iteration + 1;
    toler = max(abs(abs(V) - abs(Vprev)));
    Vprev = V; 
end 

## Example 9.1: form_Y_bus.sce

In [ ]:
clear;
clc

y=[0 1 (-.4*%i)
0 2 (-.3*%i)
1 2 (-.5*%i)]
n=2
e=3

Y=zeros(n,n)

for i=1:e
    n1=real(y(i,1))
    n2=real(y(i,2))
    ynew=y(i,3)
    if(n1==0)
        Y(n2,n2)=Y(n2,n2)+ynew
    else
        Y(n1,n1)= Y(n1,n1)+ynew
        Y(n1,n2)= Y(n1,n2)-ynew
        Y(n2,n1)= Y(n2,n1)-ynew
        Y(n2,n2)= Y(n2,n2)+ynew
    end
end
disp(Y)

## Example 9.3: form_Y_bus_and_effect_of_adding_a_line.sce

In [ ]:
clear
clc

data=[ 1 2 .025 .1
 2 3 .02 .08
  3 4 .05 .2
   1 4 .04 .16
]

n=4
e=4
Y=zeros(n,n)

for i=1:e
    ynew = 1/(data(i,3)+(%i *real(data(i,4))))
    n1=real(data(i,1))
    n2=real(data(i,2))
    if(n1==0)
        Y(n2,n2)=Y(n2,n2)+ynew
    else
        Y(n1,n1)= Y(n1,n1)+ynew
        Y(n1,n2)= Y(n1,n2)-ynew
        Y(n2,n1)= Y(n2,n1)-ynew
        Y(n2,n2)= Y(n2,n2)+ynew
    end
end

disp(round(Y*1e2)/100,'(a)')

data(e+1,:)=[1 3 .1 .4]
i=e+1
ynew = 1/(data(i,3)+(%i *real(data(i,4))))

n1=real(data(i,1))
n2=real(data(i,2))
if(n1==0)
    Y(n2,n2)=Y(n2,n2)+ynew
else
    Y(n1,n1)= Y(n1,n1)+ynew
    Y(n1,n2)= Y(n1,n2)-ynew
    Y(n2,n1)= Y(n2,n1)-ynew
    Y(n2,n2)= Y(n2,n2)+ynew
end


disp(round(Y*1e2)/100,'(b)')

## Example 9.4: find_y_bus.sce

In [ ]:
clear
clc

data=[ 1 2 .2+%i*.8 %i*.02
2 3 .3+%i*.9 %i*.03
2 4 .25+%i*1 %i*.04
3 4 .2+%i*.8 %i*.02
1 3 .1+%i*.4 %i*.01
]

n=4
e=5
Y=zeros(n,n)

for i=1:e
    ynew = 1/(data(i,3))
    y0=data(i,4)
    n1=real(data(i,1))
    n2=real(data(i,2))
    if(n1==0)
        Y(n2,n2)=Y(n2,n2)+ynew+y0
    else
        Y(n1,n1)= Y(n1,n1)+ynew+y0
        Y(n1,n2)= Y(n1,n2)-ynew
        Y(n2,n1)= Y(n2,n1)-ynew
        Y(n2,n2)= Y(n2,n2)+ynew+y0
    end
end

disp(fix(Y*1e3)/1e3)

## Example 9.5: find_missing_elements_of_y_bus.sce

In [ ]:
clear
clc

Y=[
.7-%i*3     -.2+%i   -.5+2*%i          %inf
%inf               %inf    -.3+2*%i       -.5+3*%i
%inf               %inf        %inf          -1+4*%i
%inf               %inf        %inf              %inf
]
disp('inf shows that this value is to be found ')
disp(Y,'given')

Y(1,4)=round(Y(1,1)+Y(1,3)+Y(1,2))
Y(4,4)=0-Y(1,4)-Y(2,4)-Y(3,4)
Y(4,1)=Y(1,4)
Y(2,1)=Y(1,2)
Y(3,2)=Y(2,3)
Y(3,1)=Y(1,3)
Y(4,2)=Y(2,4)
Y(4,3)=Y(3,4)
Y(2,2)=0-Y(2,1)-Y(2,4)-Y(2,3)
Y(3,3)=0-Y(3,1)-Y(3,4)-Y(3,2)



disp(Y,'completed')

## Example 9.7: find_y_bus_with_mutual_coupling_of_lines_present.sce

In [ ]:
clear
clc

z=[
%i*1    0   0   0   0
0       %i*.4   %i*.2 0 0 
0       %i*.2   %i*.5 0 0 
   0   0   0 %i*.2    0
0   0   0   0 %i*.25
]

y=inv(z)

A=[
1 0 0 0 
-1 1 0 0 
-1 0 1 0
0 0 1 -1
0 -1 0 -1
]

Y=A'*y*A

disp(Y)

## Example 9.8: find_reactive_power_generations_losses_and_powers_transferred.sce

In [ ]:
clear
clc

G=[
%inf %inf 1.5 1
0    %inf 2.0 .5
3.5  %inf 0   0
0    %inf 2  .8
]


ld=[
1 2 %i*.2
2 3 %i*.1
3 4 %i*.1
1 4 %i*.2
1 3 %i*.2
]

n=4
e=5
Y=zeros(n,n)

for i=1:e
    ynew = 1/(ld(i,3))
    y0=0
    n1=real(ld(i,1))
    n2=real(ld(i,2))
    if(n1==0)
        Y(n2,n2)=Y(n2,n2)+ynew+y0
    else
        Y(n1,n1)= Y(n1,n1)+ynew+y0
        Y(n1,n2)= Y(n1,n2)-ynew
        Y(n2,n1)= Y(n2,n1)-ynew
        Y(n2,n2)= Y(n2,n2)+ynew+y0
    end
end

G(1,1)= G(1,3)+G(2,3)+G(3,3)+G(4,3)-(G(2,1)+G(3,1)+G(4,1))

G(1,5)=G(1,1)-G(1,3)
G(2,5)=G(2,1)-G(2,3)
G(3,5)=G(3,1)-G(3,3)
G(4,5)=G(4,1)-G(4,3)

//d1=0
A=%i * [
Y(2,2) Y(2,3) Y(2,4)
Y(3,2) Y(3,3) Y(3,4)
Y(4,2) Y(4,3) Y(4,4)
]
d(1)=0
d(2:4)=inv(A) * [G(2,5) G(3,5) G(4,5)]'
d=round((d)*1000)/1000
G(1,6)= %i *((Y(1,1) * cos(d(1)-d(1)))+(Y(1,2) *  cos(d(1)-d(2)))+(Y(1,3) * cos(d(1)-d(3)) )+(Y(1,4) *  cos(d(1)-d(4))));
G(2,6)= %i *((Y(2,1) * cos(d(2)-d(1)))+(Y(2,2) *  cos(d(2)-d(2)))+(Y(2,3) * cos(d(2)-d(3)) )+(Y(2,4) *  cos(d(2)-d(4))));
G(3,6)= %i *((Y(3,1) * cos(d(3)-d(1)))+(Y(3,2) *  cos(d(3)-d(2)))+(Y(3,3) * cos(d(3)-d(3)) )+(Y(3,4) *  cos(d(3)-d(4))));
G(4,6)= %i *((Y(4,1) * cos(d(4)-d(1)))+(Y(4,2) *  cos(d(4)-d(2)))+(Y(4,3) * cos(d(4)-d(3)) )+(Y(4,4) *  cos(d(4)-d(4))));

G(:,6)=round(G(:,6) *10000)/10000
G(:,2)=G(:,6) + G(:,4)
G(:,2)=round(G(:,2)*1e4)/1e4
mprintf('
(a) Q1= %.4f, Q2= %.4f, Q3= %.3f, Q4= %.4f, ', G(1,2),G(2,2),G(3,2),G(4,2))

Ql=G(1,6)+G(2,6)+G(3,6)+G(4,6)
mprintf('
(b) reactive line losses=%.4fpu',Ql)

X=ld(:,3)/%i

P(1)=round((sin(d(1)-d(2))/X(1))*1000)/1000
P(2)=round((sin(d(2)-d(3))/X(2))*1000)/1000
P(3)=round((sin(d(3)-d(4))/X(3))*1000)/1000
P(4)=round((sin(d(1)-d(4))/X(4))*1000)/1000
P(5)=round((sin(d(1)-d(3))/X(5))*1000)/1000

Q(1)=round(((1-cos(d(1)-d(2)))/X(1))*10000)/10000
Q(2)=round(((1-cos(d(2)-d(3)))/X(2))*10000)/10000
Q(3)=round(((1-cos(d(3)-d(4)))/X(3))*10000)/10000
Q(4)=round(((1-cos(d(1)-d(4)))/X(4))*10000)/10000
Q(5)=round(((1-cos(d(1)-d(3)))/X(5))*10000)/10000

mprintf('
(c)')
mprintf('
P12 = -P21 = %.3f, Q12=Q21=%.4f',P(1),Q(1))
mprintf('
P23 = -P32 = %.3f, Q23=Q32=%.3f',P(2),Q(2))
mprintf('
P34 = -P43 = %.3f, Q34=Q43=%.3f',P(3),Q(3))
mprintf('
P14 = -P41 = %.3f, Q14=Q41=%.4f',P(4),Q(4))
mprintf('
P13 = -P31 = %.3f, Q13=Q31=%.4f',P(5),Q(5))

## Example 9.9: solve_using_gauss_seidel_for_1_variable.sce

In [ ]:
clear
clc

e=1e-5
x=1
E=100
while (E>e)
    x1=.5*(7+log10(x))
    E=abs(x-x1)
    x=x1
end

mprintf('x = %f',x)